# Edge detection Algorithms

<div style="text-align: center;">
    Víctor Vega Sobral
</div>

In this notebook, I´ll implement the canny edge detection algorithm to three different images of Formula 1 cars. The content of the notebook will be: 

* Define the code of the algorithm.
* Apply it to the three different images.
* Experiment with the parameters to see the different results.
* Analysis and conclussions with:
    - Quality of detected edges.
    - Computational performance (execution time).
    - Sensitivity to noise.
    - Effectiveness under different lightning conditions.
    
---

In [4]:
__author__ = "Víctor Vega Sobral"

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time


## Edge Detection Algorithms implementation: first steps

First, we need to create a function to show the images on a Jupyter Notebook

--- 

In [6]:
def show_image(title, image, cmap=None):
    plt.figure(figsize=(6,6))
    plt.title(title)
    if cmap:
        plt.imshow(image, cmap=cmap)
    else:
        # Covert from BGR to RGB
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()


### Loading the images to our notebook

---

In [ ]:
# Paths to the 3 images 

image_paths = {
    "well_defined": "../TASK 1/edges.jpg",       # defined edges
    "complex_texture": "../TASK 1/complex.jpg", # complex textures
    "low_contrast": "../TASK 1/low_contrast.jpg"          # low contrast
}

images = {}
for key, path in image_paths.items():
    img = cv2.imread(path)
    if img is None:
        print(f"Error al cargar la imagen: {path}")
    else:
        images[key] = img


### Preprocessing the images

We need to convert the images to grayscale and blur them. For this, I´ll use this functions:

1. `cv2.COLOR_BGR2GRAY`: converts the image to grayscale.
2. `cv2.GaussianBlur`: applies Gaussian blur.


---

In [ ]:
def preprocess(image, blur_kernel = (5,5)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, blur_kernel, 0)
    return gray, blurred

### Edge Detection Implementations.

In the following cells, three detection algorithms are implemented.

#### Canny edge

In [ ]:
def canny_edge(image, lower = 10, upper = 200):
    return cv2.Canny(image, lower, upper)

#### Sobel edge

In [ ]:
def sobel_edge(image, ksize = 5):
    # We need to calculate the x and y derivatives
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize= ksize)
    sobely = cv2.Sovel(image, cv2.CV_64F, 0, 1 , ksize =ksize)
    sobel = np.sqrt(sobelx**2 + sobely **2)

    # Convert to 8 bits

    sobel = np.uint8(np.clip(sobel, 0, 255))

    return sobel

#### Laplacian edge 

In [ ]:
def laplacian_edge(image, ksize = 3):
    laplacian = cv2.Laplacian(image, cv2.CV_64F, ksize=ksize)
    laplacian = np.uint8(np.clip(np.abs(laplacian), 0, 255))
    return laplacian

### Adjusting the parameters

In this cell, I provide the code to change and experiment with different parameters.

1. First parameters used: 

    ``python
    canny_params = {"lower": 30, "upper": 150}
    sobel_params = {"ksize": 5}
    laplacian_params = {"ksize": 3}
    ``



---

In [10]:
# Adjustable parameters
canny_params = {"lower": 30, "upper": 150}
sobel_params = {"ksize": 5}
laplacian_params = {"ksize": 3}

### Preprocessing the images

---

In [ ]:
image_key = "well_defined"
orig = images[image_key]
gray, blurred = preprocess(orig)

In [ ]:
image_key = "complex_texture"
orig = images[image_key]
gray, blurred = preprocess(orig)

In [ ]:
image_key = "low_contrast"
orig = images[image_key]
gray, blurred = preprocess(orig)

### Applying the algorithms

---

#### Canny

In [ ]:
edges_canny = canny_edge(blurred, **canny_params)

#### Sobel


In [ ]:
edges_sobel = sobel_edge(blurred, **sobel_params)

#### Laplacian

In [ ]:
edges_laplacian = laplacian_edge(blurred, **laplacian_params)

## Results

---

In [ ]:
show_image("Original images", orig)


In [ ]:
show_image("Canny", edges_canny, cmap="gray")


In [ ]:
show_image("Sobel", edges_sobel, cmap="gray")


In [ ]:
show_image("Laplacian", edges_laplacian, cmap="gray")